In [25]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [26]:
X,y = make_classification(n_samples=10000, n_features=10,n_informative=3)

In [27]:
X

array([[-1.18050951e+00,  1.83461894e+00, -2.18864067e+00, ...,
         8.00078550e-01,  9.55386835e-01,  1.43236172e-01],
       [ 8.38784676e-01, -8.22975814e-01, -2.98690896e+00, ...,
         2.89441503e+00,  7.37765239e-01,  2.13060604e+00],
       [ 2.03413220e+00, -9.06939194e-01, -6.26147848e-01, ...,
        -3.35204070e-01,  2.82291452e-01, -7.52160578e-01],
       ...,
       [ 7.30622052e-01,  6.62534820e-01, -3.08437718e-01, ...,
        -6.50505629e-01,  2.20763198e-03, -1.19075589e+00],
       [ 7.13017629e-01,  1.31388430e-02,  8.09187975e-01, ...,
         1.05817927e+00, -9.19507232e-01,  1.81294016e+00],
       [ 1.58565041e-01, -9.37048347e-01, -2.62263495e+00, ...,
         7.44408005e-01, -1.32746606e+00, -2.58101058e-01]])

In [28]:
y

array([1, 1, 1, ..., 0, 1, 1])

In [29]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [30]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)

print("Decision Tree accuracy",accuracy_score(y_test,y_pred))

Decision Tree accuracy 0.856


In [31]:
y_pred

array([0, 1, 0, ..., 1, 1, 1])

# Bagging

In [32]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.5,
    bootstrap=True,
    random_state=42
)

In [33]:
bag.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [34]:
y_pred = bag.predict(X_test)
y_pred

array([0, 1, 1, ..., 1, 0, 1])

In [35]:
accuracy_score(y_test,y_pred)

0.91

In [36]:
# 4000 cz I took  max_samples=0.5 that means 8000/2 = 4000

bag.estimators_samples_[0].shape

(4000,)

In [37]:
bag.estimators_features_[0].shape

(10,)

# Bagging using SVM

In [38]:
bag = BaggingClassifier(
    base_estimator=SVC(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    random_state=42
)

In [39]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
accuracy_score(y_test,y_pred)


0.8705

In [40]:
y_pred

array([0, 1, 1, ..., 1, 0, 0])

# Pasting

In [41]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False,
    random_state=42,
    verbose = 1,  # training time all info show korbe.
    n_jobs=-1     # task Devide hoye jabe basically Ektu fast hoye jabe.
    
)

In [42]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Pasting classifier",accuracy_score(y_test,y_pred))

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    7.8s remaining:   23.5s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    8.6s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.1s remaining:    0.5s


Pasting classifier 0.905


[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.4s finished


# Random Subspaces

In [43]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,    # full row niye astechi 
    bootstrap=False,
    max_features=0.5,    #  Column 5 ta Niye kaj korbe. Not more than that.
    bootstrap_features=True,
    random_state=42
)

In [44]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Subspaces classifier",accuracy_score(y_test,y_pred))

Random Subspaces classifier 0.89


In [45]:
bag.estimators_samples_[0].shape

(8000,)

In [46]:
bag.estimators_features_[0].shape

(5,)

# Random Patches

In [47]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [48]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Patches classifier",accuracy_score(y_test,y_pred))

Random Patches classifier 0.8875


# OOB Score
###  out of bag samples- jokhon row sampleing kora hoy tokhon kichu row thake jegulo kono D.tree pay na. kichu Multiple row thakte pare ja bar bar D.tree paite pare. 37% vetore theke jay ja D.tree pay na. Ar ei 37% kei OOB bole. [statistically proven]

In [49]:
bag = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)

In [50]:
bag.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [51]:
bag.oob_score_

0.9035

In [52]:
y_pred = bag.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))

Accuracy 0.905


# Applying GridSearchCV ( Best Accuricy )

In [57]:
from sklearn.model_selection import GridSearchCV

In [58]:
parameters = {
    'n_estimators': [50,100,500], 
    'max_samples': [0.1,0.4,0.7,1.0],
    'bootstrap' : [True,False],
    'max_features' : [0.1,0.4,0.7,1.0]
    }

In [59]:
search = GridSearchCV(BaggingClassifier(), parameters, cv=5)

In [ ]:
search.fit(X_train,y_train)

In [ ]:
search.best_params_
search.best_score_

In [ ]:
search.best_params_